In [ ]:
%pip install -qU langchain-openai langchain langgraph langchain_community langgraph-supervisor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 930.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.4/65.4 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.4/152.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 28.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.1/438.1 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.0/363.0 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 8.3 MB/s eta 0:00:00


In [ ]:
from pydantic import BaseModel
from langchain_openai import ChatOpenAI
from google.colab import userdata
from langgraph.prebuilt import create_react_agent
from langgraph_supervisor import create_supervisor
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage,
)
from langgraph.prebuilt import create_react_agent
import requests
key = userdata.get('OPENAI_API_KEY')

In [ ]:
model = ChatOpenAI(api_key=key,model="gpt-4o-mini")

In [ ]:
NOTION_TOKEN = "key here"
DATABASE_ID = "20f4296aca078044a20fd88ec8246774"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

# Prompts

In [ ]:
#original
# delegate_agent_prompt = """

# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent

# - Only run “notion_page_agent” function when user explicitly states to create a Notion page

# - When keyword phrase "in my notion page” is detected, transfer to Notion Block Agent. For example "Create a bullet list from “Chapter 1" to
# “Chapter 11" in my notion page".
# """

In [ ]:
#2nd try
# delegate_agent_prompt = """

# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent

# - Only run “notion_page_agent” function when user explicitly states to create a Notion page
# - If the user requests to "create a new page with content", first run `notion_page_agent` to create the page.
# - Then pass the page ID and content to `notion_block_agent` to insert it as blocks.

# - When keyword phrase "in my notion page” is detected, transfer to Notion Block Agent. For example "Create a bullet list from “Chapter 1" to
# “Chapter 11" in my notion page".
# """

In [ ]:
# #3rd try
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# ## TASK ROUTING RULES

# 1. Only run `notion_page_agent` function when the user explicitly states to create a Notion page.
# 2. If the user requests to "create a new page with content":
#     - First, call `notion_page_agent` to create the page and retrieve the page_id.
#     - Then, pass the retrieved `page_id` and the requested content to `notion_block_agent` to insert the content as blocks.

# 3. When the keyword phrase "in my notion page" is detected:
#     - This implies content should be written into a Notion page.
#     - If a page was just created, use its `page_id`.
#     - If not, ask the user to specify the page or title, then call `notion_block_agent` with the `page_id` and content.

# 4. Ensure the page_id returned by `notion_page_agent` is captured and passed to `notion_block_agent` when adding content to the new page.

# 5. If the user provides a string list (e.g. "Chapter 1, Chapter 2"), ensure it is split correctly before passing to `notion_block_agent`.

# ## Examples of correct behavior:

# - User says: "Create a new page named Bankai2."
#     - Call `notion_page_agent` with title "Bankai2".

# - User says: "Create a new page named Bankai2. Write about anime Bleach in my notion page."
#     - Step 1: Call `notion_page_agent(title='Bankai2')`
#     - Step 2: Capture `page_id` from response.
#     - Step 3: Call `notion_block_agent(page_id=..., content='Write about anime Bleach')`

# ## TOOL CHAINING BEHAVIOR

# - After calling `notion_page_agent`, capture the `page_id` returned in its response.
# - Then call `notion_block_agent` with:
#   - `page_id`: the ID from the page creation step
#   - `content`: the remaining content user requested (e.g., "write about anime bleach")

# Use tool calls with dependency if supported (e.g., tool calling followed by continuation).
# If tool call chaining isn't possible, return a ToolMessage after `notion_page_agent` with the `page_id`, and ensure the supervisor continues with `notion_block_agent`.

# ## Error Handling:
# - If the `page_id` is missing when trying to insert blocks, respond with: "Missing required information: page_id"
# - If tool is not available to perform the action, respond with: "I am unable to perform the request"
# """


In [ ]:
# #4th prompt
# delegate_agent_prompt = """
# You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

# ## TASK FLOW

# 1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

# 2. If the user also wants to "write about ..." in the same request:
#     - After calling `notion_page_agent`, extract the `page_id` from its response.
#     - Then call `notion_block_agent(page_id=..., content=...)`.

# ## PAGE_ID EXTRACTION

# - When `notion_page_agent` returns, it includes a URL like `https://www.notion.so/<title>-<page_id>`.
# - Extract the final part of the URL (`<page_id>`) and use it in the next step.

# ## EXAMPLE:

# If user says:
# "Create a new page named Ichigo. write about anime bleach in my notion page."

# You must:

# 1. Call `notion_page_agent(title='Ichigo')`
# 2. Wait for response, extract `page_id = "2104296aca0781f08ef8caf4bf9ad809"` from the URL.
# 3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

# ---

# If you cannot extract the `page_id`, ask the user to specify or confirm the page.
# """


In [ ]:
#5th prompt
delegate_agent_prompt = """
You are a Notion agent designed to interact with the Notion API. Your primary task is to delegate tasks to the appropriate agent.

## TASK FLOW

1. If the user says "Create a new page named <title>", call `notion_page_agent(title=...)`.

2. If the user also wants to "write about ..." in the same request:
    - First, call `notion_page_agent(title=...)`.
    - The `notion_page_agent` internally uses a tool called `extract_page_id_from_notion_response` to extract the `page_id` from the response.
    - Once `page_id` is returned by `notion_page_agent`, call `notion_block_agent(page_id=..., content=...)`.

## PAGE_ID EXTRACTION

- The `notion_page_agent` tool will return the `page_id` directly using `extract_page_id_from_notion_response`.
- You do not need to parse the URL manually anymore. Use the `page_id` value returned from `notion_page_agent`.

## EXAMPLE:

If the user says:
"Create a new page named Ichigo. write about anime bleach in my notion page."

You must:

1. Call `notion_page_agent(title='Ichigo')`
2. Receive the response: `page_id = "2104296aca0781f08ef8caf4bf9ad809"`
3. Then call: `notion_block_agent(page_id='2104296aca0781f08ef8caf4bf9ad809', content='write about anime bleach')`

---

If `notion_page_agent` fails to return a `page_id`, ask the user to specify or confirm the page manually.
"""


In [ ]:
page_agent_prompt = """
You are a Notion agent responsible for managing pages within a Notion workspace. You're equipped with different tools.

Your primary tasks include:
1. **Creating Pages**: Create new pages with specified titles, content, and properties. Ensure pages are formatted according to user requests.

When performing actions:
- Always ensure page titles and content are clear and concise.
- If tool is not available to perform the action, simply say “I am unable to perform the request"
"""

In [ ]:
block_agent_prompt = """
You are a Notion agent responsible for add blocks within Notion pages. Your tasks include:
    - **Creating Blocks**: Add new blocks to a page. Ensure the blocks are created with the proper content and formatting as specified by the user.

2. FORMATTING REQUIREMENTS:
  - Follow Notion's native styling
  - Maintain consistent spacing
  - Preserve hierarchical structure
  - Apply appropriate indentation

## ERROR HANDLING:
  - If a requested block type is unsupported: Respond with ‘Block type not supported’
  - If missing required parameters: Respond with ‘Missing required information: [parameter]’
  - For unauthorized actions: Respond with ‘Permission denied’
  - If tool is not available to perform the action: Respond with ‘I am unable to perform the request’

## Important:
  - When using add_notion_number_list_block, add_notion_bulleted_list_block, and add_notion_to_do_block functions, argument must be a
      list of str. For example
      add_notion_number_list_block([‘item 1°, ‘item 2‘, ‘item 3'])
      add_notion_bulleted_list_block([‘item 1', ‘item 2', ‘item 3'])
      add_notion_to_do_block([‘item 1', ‘item 2', ‘item 3'])
  - When passing a string containing a list of item, Use comma (",") as separator.
    For example, “Chapter 1, Chapter 2, "Chapter 3" is acceptable. “Chapter 1\nChapter 2\n Chapter 3" is not.
"""

# Tools

In [ ]:
import os
from typing import List
import httpx
from langchain_core.tools import tool

NOTION_API_KEY = "key here"
NOTION_ENDPOINT = 'https://api.notion.com/v1'
NOTION_PAGE_ID='20f4296aca078079949feb0f4dfa1d48'

@tool
def create_notion_page(page_title):
  """
  Create a new page in Notion with the specified title and content.

  args:
      page_title: The title of the new page.
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/pages'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  properties= {
        "title": [
            {
                "text": {
                    "content": page_title
                }
            }
        ]
    }
  try:
    response = httpx.post(url, headers=headers, json={
        'parent': {
            'page_id': page_id
        },
        'properties': properties
    })
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}





In [ ]:
@tool
def add_notion_heading_block(
        heading_type: str = 'heading_1',
        content: str = 'New Heading',
        is_hyperlink: bool = False,
        hyperlink_url: str = '',
        color: str = 'default'
):
  """
  Add a heading block to a Notion page with the specified content.

  args:
      context_variables: The context variables containing the API key and page ID.
      heading_type: The type of heading block to add (e.g., "heading_1', 'heading_2").
      content: The content of the heading block.
      is_hyperlink: A boolean value indicating whether the heading should be a hyperlink.
      hyperlink_url: The URL of the hyperlink.
      color: The color of the heading block.

  example usage:
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=False)
      add_notion_heading_block (heading_type='heading_1', content='Lacinato Kale', is_hyperlink=True, hyperlink_url='https://en.wikipedia.org/wiki/Lacinato_kale')
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  text_obj = {'type': 'text', 'text': {'content': content}}
  data = {
      'children': [{
          'object': 'block',
          'type': heading_type,
          heading_type: {
              'rich_text': [text_obj],
              'color': color # Move color inside the heading block
          }
      }]
  }

  if is_hyperlink:
    text_obj['text']['link'] = {'url': hyperlink_url}

  try:
    response = httpx.patch(url, headers=headers, json=data)
    return response.json()
  except httpx.ReadTimeout:
    return {'error': 'Request timed out'}

In [ ]:
#global page id
# tool
# def add_notion_paragraph_block(content: str, is_hyperlink: bool = False, hyperlink_url: str = '', color: str = 'default'):
#     """
#     Add a paragraph block to a Notion page with the specified content.

#     Args:
#         content: The content of the paragraph block.
#         is_hyperlink: Whether the paragraph is a hyperlink.
#         hyperlink_url: The URL if it’s a hyperlink.
#         color: The color of the paragraph block.

#     Example usage:
#         add_notion_paragraph_block(content='Hello', is_hyperlink=False)
#         add_notion_paragraph_block(content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')
#     """
#     url = f'{NOTION_ENDPOINT}/blocks/{NOTION_PAGE_ID}/children'
#     headers = {
#         'Authorization': f'Bearer {NOTION_API_KEY}',
#         'Content-Type': 'application/json',
#         'Notion-Version': '2022-06-28'
#     }

#     text_obj = {'type': 'text', 'text': {'content': content}}
#     if is_hyperlink:
#         text_obj['text']['link'] = {'url': hyperlink_url}

#     data = {
#         'children': [{
#             'object': 'block',
#             'type': 'paragraph',
#             'paragraph': {
#                 'rich_text': [text_obj],
#                 'color': color
#             }
#         }]
#     }

#     try:
#         response = httpx.patch(url, headers=headers, json=data)
#         return response.json()
#     except httpx.ReadTimeout:
#         return {'error': 'Request timed out'}


In [ ]:
@tool
def add_notion_paragraph_block(
    page_id: str,
    content: str,
    is_hyperlink: bool = False,
    hyperlink_url: str = '',
    color: str = 'default'
):
    """
    Add a paragraph block to a Notion page with the specified content.

    Args:
        page_id: The ID of the Notion page where the block will be added.
        content: The content of the paragraph block.
        is_hyperlink: Whether the paragraph is a hyperlink.
        hyperlink_url: The URL if it’s a hyperlink.
        color: The color of the paragraph block.

    Example usage:
        add_notion_paragraph_block(page_id='abc123', content='Hello', is_hyperlink=False)
        add_notion_paragraph_block(page_id='abc123', content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')
    """
    url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
    headers = {
        'Authorization': f'Bearer {NOTION_API_KEY}',
        'Content-Type': 'application/json',
        'Notion-Version': '2022-06-28'
    }

    text_obj = {'type': 'text', 'text': {'content': content}}
    if is_hyperlink:
        text_obj['text']['link'] = {'url': hyperlink_url}

    data = {
        'children': [{
            'object': 'block',
            'type': 'paragraph',
            'paragraph': {
                'rich_text': [text_obj],
                'color': color
            }
        }]
    }

    try:
        response = httpx.patch(url, headers=headers, json=data)
        return response.json()
    except httpx.ReadTimeout:
        return {'error': 'Request timed out'}


In [ ]:
add_notion_paragraph_block(page_id="2104296aca07810c9976e4d7f3ee544f", content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')

{'object': 'list',
 'results': [{'object': 'block',
   'id': '2114296a-ca07-814b-bcdb-cc90e7c989d4',
   'parent': {'type': 'page_id',
    'page_id': '2104296a-ca07-810c-9976-e4d7f3ee544f'},
   'created_time': '2025-06-13T03:17:00.000Z',
   'last_edited_time': '2025-06-13T03:17:00.000Z',
   'created_by': {'object': 'user',
    'id': '774be3bb-5f8b-4662-a478-0dbd3fe7c835'},
   'last_edited_by': {'object': 'user',
    'id': '774be3bb-5f8b-4662-a478-0dbd3fe7c835'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [{'type': 'text',
      'text': {'content': 'Click me', 'link': {'url': 'https://example.com/'}},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'default'},
      'plain_text': 'Click me',
      'href': 'https://example.com/'}],
    'color': 'default'}}],
 'next_cursor': None,
 'has_more': False,
 '

In [ ]:
sample_response = "I have created a new page titled **Orihime** in your Notion workspace.  Additionally, I have created another page with the title **Orihime Inoue Description**.\n\n You can find the details in the pages linked below:\n\n - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

page_id = extract_page_id_from_notion_response(sample_response)
print(f"Extracted page id: {page_id}")
add_notion_paragraph_block(page_id=page_id, content='Click me', is_hyperlink=True, hyperlink_url='https://example.com')

Extracted page id: 2104296aca0781e299b5d1e16f714891


{'object': 'list',
 'results': [{'object': 'block',
   'id': '2114296a-ca07-81ee-84da-ebe0524e9751',
   'parent': {'type': 'page_id',
    'page_id': '2104296a-ca07-81e2-99b5-d1e16f714891'},
   'created_time': '2025-06-13T03:19:00.000Z',
   'last_edited_time': '2025-06-13T03:19:00.000Z',
   'created_by': {'object': 'user',
    'id': '774be3bb-5f8b-4662-a478-0dbd3fe7c835'},
   'last_edited_by': {'object': 'user',
    'id': '774be3bb-5f8b-4662-a478-0dbd3fe7c835'},
   'has_children': False,
   'archived': False,
   'in_trash': False,
   'type': 'paragraph',
   'paragraph': {'rich_text': [{'type': 'text',
      'text': {'content': 'Click me', 'link': {'url': 'https://example.com/'}},
      'annotations': {'bold': False,
       'italic': False,
       'strikethrough': False,
       'underline': False,
       'code': False,
       'color': 'default'},
      'plain_text': 'Click me',
      'href': 'https://example.com/'}],
    'color': 'default'}}],
 'next_cursor': None,
 'has_more': False,
 '

In [ ]:
@tool
def add_notion_code_block(code: str, Language: str = 'python', caption: List[str] = []):
  """
  Add a code block to a Notion page with the speccified code content.

  args:
    code: The code content of the code block.
    language: The programming language of the code block.
    caption: A list of strings for the caption of the code block.
  """
  api_key = NOTION_API_KEY
  page_id = NOTION_PAGE_ID
  url = f'{NOTION_ENDPOINT}/blocks/{page_id}/children'
  headers = {
      'Authorization': f'Bearer {api_key}',
      'Content-Type': 'application/json',
      'Notion-Version': '2022-06-28'
  }
  data = {
      'children': [{
          'object': 'block',
          'type': 'code',
          'code': {
              'caption': [{'text': {'content': c}} for c in caption],
              'rich_text': [{'type': 'text', 'text': {'content': code}}],
              'language': Language
          }
      }]
  }

  try:
    response = httpx.patch (url, headers-headers, json=data)
    return response.json()
  except httpx. ReadTimeout:
    return {'error': 'Request timed out'}

In [ ]:
!pip install -q notion-client

In [ ]:
from notion_client import Client

import re
from langchain.tools import tool


def extract_page_id_from_notion_response(response: str) -> str:
    """
    Extracts and returns the page id from a Notion URL within the given text.

    The function searches for a pattern matching a Notion URL that looks like:
      https://www.notion.so/<PageTitle>-<page_id>

    For example, given a response like:
      "I have created a new page titled **Orihime** in your Notion workspace. Additionally, I have created another page with the title **Orihime Inoue Description**.

       You can find the details in the pages linked below:

       - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

    This function will return the page id:
      "2104296aca0781e299b5d1e16f714891"

    If no matching page id is found, it returns an explanatory message.
    """
    # This regex looks for a URL starting with https://www.notion.so/
    # followed by any characters (representing the title) and a hyphen,
    # then a 32-character hexadecimal string (the page id).
    pattern = r"https:\/\/www\.notion\.so\/[^-\s]+-([a-f0-9]{32})"
    match = re.search(pattern, response)
    if match:
        return match.group(1)
    else:
        return "No page id found in the response."


In [ ]:
sample_response = "I have created a new page titled **Orihime** in your Notion workspace.  Additionally, I have created another page with the title **Orihime Inoue Description**.\n\n You can find the details in the pages linked below:\n\n - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"

page_id = extract_page_id_from_notion_response(sample_response)
print(f"Extracted page id: {page_id}")

Extracted page id: 2104296aca0781e299b5d1e16f714891


# Agents with Response

In [ ]:
notion_page_agent = create_react_agent(
    model=model,
    tools=[create_notion_page, extract_page_id_from_notion_response],
    prompt=page_agent_prompt,
    name="notion_page_agent"
)

In [ ]:
response = notion_page_agent.invoke(
    {
        "messages": [
            HumanMessage(content="create a new page named Bleach.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='create a new page named Bleach.', additional_kwargs={}, response_metadata={}, id='bf4506c2-b84c-4fae-a9b9-8bb8d9094195'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ysl31Q92bclJgxoFPsxChjO2', 'function': {'arguments': '{"page_title":"Bleach"}', 'name': 'create_notion_page'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 168, 'total_tokens': 186, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhV2zPeyXwOOY8bqSSsKeB7nieji1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='notion_page_agent', id='run--5a17b0aa-3479-4606-8ce8-e4ed528f4dc1-0', tool_calls=[{'name': 'create_notion_page', 'a

In [ ]:
response["messages"][-1].content

'A new page named **Bleach** has been successfully created. You can view it [here](https://www.notion.so/Bleach-2104296aca07814bb938dcbf885316e7).'

In [ ]:
notion_block_agent = create_react_agent(
    model=model,
    tools=[add_notion_heading_block, add_notion_paragraph_block, add_notion_code_block, extract_page_id_from_notion_response],
    prompt=block_agent_prompt,
    name="notion_block_agent"
)

In [ ]:
response = notion_block_agent.invoke(
    {
        "messages": [
            HumanMessage(content="write a paragraph on anime named Naruto. get the page id from this response: I have created a new page titled **Orihime** in your Notion workspace.  Additionally, I have created another page with the title **Orihime Inoue Description**.\n\n You can find the details in the pages linked below:\n\n - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)")
            ]
    }
    )
response["messages"]

[HumanMessage(content='write a paragraph on anime named Naruto. get the page id from this response: I have created a new page titled **Orihime** in your Notion workspace.  Additionally, I have created another page with the title **Orihime Inoue Description**.\n\n You can find the details in the pages linked below:\n\n - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)', additional_kwargs={}, response_metadata={}, id='61093ec9-8d3f-40d8-af8b-356fb2346658'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Q64yw1kmuUk31ZgDipdPYbq5', 'function': {'arguments': '{"response":"I have created a new page titled **Orihime** in your Notion workspace.  Additionally, I have created another page with the title **Orihime Inoue Description**.\\n\\n You can find the details in the pages linked below:\\n\\n - [Orihime](https://www.notion.so/Orihime-2104296aca0781e299b5d1e16f714891)"}', 'name': 'extract_page_id_from_notion_response'}, 'type': 'function'}], 'r

In [ ]:
response["messages"][-1].content

'A paragraph about the anime "Naruto" has been successfully added to your Notion page. If you need any further assistance or additional information, feel free to ask!'

In [ ]:
response = notion_block_agent.invoke(
    {
        "messages": [
            HumanMessage(content="write a paragraph on anime named Bleach.")
            ]
    }
    )
response["messages"]

[HumanMessage(content='write a paragraph on anime named Bleach.', additional_kwargs={}, response_metadata={}, id='7cd0acd1-4332-4bcb-af14-683360b16594'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_vSocssdXCBOFjYECrDClaQmp', 'function': {'arguments': '{"content":"Bleach is a popular anime series created by Tite Kubo, which follows the adventures of Ichigo Kurosaki, a teenager who becomes a Soul Reaper after gaining powers from Rukia Kuchiki. The story unfolds as Ichigo takes on the responsibility of protecting the living world from evil spirits known as Hollows, while also guiding the souls of the deceased to the afterlife. The series intricately weaves themes of friendship, sacrifice, and the struggle between good and evil, featuring a dynamic cast of characters and intense battles. Since its debut in 2004, Bleach has garnered a massive fan following and remains influential in the anime community.","is_hyperlink":false}', 'name': 'add_notion_paragraph_block'}

In [ ]:
response["messages"][-1].content

'The paragraph on the anime "Bleach" has been successfully added to the Notion page. If you need any further assistance or additional information, feel free to ask!'

# Supervisor Agent

In [ ]:
supervisor = create_supervisor(
    agents=[notion_page_agent, notion_block_agent],

    model=model,
    prompt=delegate_agent_prompt
).compile()



# 1st prompt

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Bankai. and write about anime bleach inside that newly created page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Bankai. and write about anime bleach inside that newly created page.', additional_kwargs={}, response_metadata={}, id='919670dd-2108-4b09-9bd2-63fc51f7da79'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_0kM0CNXIKwc6KchZrFCVshdb', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 183, 'total_tokens': 197, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhV6jE5VAfvFHeLw4OZmqaxDZzrJF', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--d55057a7-b5d5-4

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Bankai2. and write about anime bleach in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Bankai2. and write about anime bleach in my notion page.', additional_kwargs={}, response_metadata={}, id='de68b5af-1b84-4428-b1e0-8f7610124271'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yOWJoMsAKaWcnDOfQUeCpvOR', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 231, 'total_tokens': 245, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhWa3iGfDFgQsuuu067v9x1LJRGr1', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--4ad03baf-9d91-43f1-8cb3-c58

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="write about anime bleach in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='write about anime bleach in my notion page.', additional_kwargs={}, response_metadata={}, id='0381b82c-4efb-4399-acc9-a31f87322f85'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_bXQTrunCeOkzqeqBgXodLlvd', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_block_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 221, 'total_tokens': 235, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhWb4rgT4CL20p3B5xlI7pCa3NEOT', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--37e46d4e-df63-4d17-872b-c578e507563f-0', tool_calls=[{'name': '

# 2 nd prompt

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Bankai2. write about anime bleach in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Bankai2. write about anime bleach in my notion page.', additional_kwargs={}, response_metadata={}, id='7c2c5f94-b52c-4697-a502-f8873da1d8d9'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Dc4unecD9pgtWD8TjEB2GZJL', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 230, 'total_tokens': 244, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BhWeOC1yL3jTRshilcHFj8pbjmyae', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--a83b74ca-b187-41e4-89fd-7e4dded

# 3rd prompt

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Ichigo. write about anime character Ichigo Kurosaki in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Ichigo. write about anime bleach in my notion page.', additional_kwargs={}, response_metadata={}, id='07acfb88-5662-4746-9adc-b8b7157c927c'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_n3B3P9GZfBEcwkDefYFvvPPI', 'function': {'arguments': '{"title":"Ichigo"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': 653, 'total_tokens': 672, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BhZVjv8VS0r8HQXVWmBFw33SrzQog', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--442bd68e-2acb-46

# 4th prompt

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Orihime. write 3 sentences about anime character Orihime Inoue in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Orihime. write 3 sentences about anime character Orihime Inoue in my notion page.', additional_kwargs={}, response_metadata={}, id='f1a6a60e-96ac-433b-9640-bf969b359ccb'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5XtR1LgpXkXhRB2KVMoeYzgp', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 662, 'total_tokens': 676, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BhZg4ksFRqdNHSTiN90zvItXV6coR', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='run--d7

# 5th prompt

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Yoruichi Shihouin. write 3 sentences about anime character Yoruichi Shihouin in my notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Yoruichi Shihouin. write 3 sentences about anime character Yoruichi Shihouin in my notion page.', additional_kwargs={}, response_metadata={}, id='193332fe-5da1-498d-8a22-7e90e3d7f778'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_X2bvUPN9VpR3Wk79jTWb2dT1', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 450, 'total_tokens': 464, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BhaOTB56sqY1kqdapkqySDLKtZ1X9', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor

# 6th attempt giving extraction function to block agent

created page and wrote inside  it

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Gin. write 3 sentences about anime character Gin Ichimaru in my notion page named Gin."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Gin. write 3 sentences about anime character Gin Ichimaru in my notion page named Gin.', additional_kwargs={}, response_metadata={}, id='dd4a1220-a19a-45d5-98db-e8aeae56a621'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ltl2AWdaPLqyhNEc1Q76dF9r', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 443, 'total_tokens': 457, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-BhqykrfKUQH4XTmPhiLByeFteIf2e', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='ru

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Chad. write 3 sentences about anime character Chad from bleach in the same notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Chad. write 3 sentences about anime character Chad from bleach in the same notion page.', additional_kwargs={}, response_metadata={}, id='2360a250-7bec-4eaf-8152-0e35d7efd6a2'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_N5vHjVhvETkSofhxMEGe67Uj', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 441, 'total_tokens': 455, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_62a23a81ef', 'id': 'chatcmpl-Bhr1wy9xzs6nIF6OTcLKR7Vd3A52J', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', id='r

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="write 3 sentences about anime character Chad from bleach in the same notion page. Create a new page named Chad 101. "
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='write 3 sentences about anime character Chad from bleach in the same notion page. Create a new page named Chad 101. ', additional_kwargs={}, response_metadata={}, id='822f98ea-23ab-43a4-be16-fff3b8acbdf5'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Ywgp0vjl6e78IfsuXUOIhiOj', 'function': {'arguments': '{"title":"Chad 101"}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 444, 'total_tokens': 465, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bj6AZ3DUKtqY3Iy6LQYy8ZUIA59xx', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, n

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Chad 102. write 3 sentences about anime character Chad from bleach in the same notion page.  "
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Chad 102. write 3 sentences about anime character Chad from bleach in the same notion page.  ', additional_kwargs={}, response_metadata={}, id='332f1d64-cdcd-47ab-9114-0c9f0eab193e'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_CXCu7An595CUjFuYWUx6WRe0', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 444, 'total_tokens': 458, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bj6BpOaGA9JdqJjGhuZfncDUnue6K', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor',

In [ ]:
for chunk in supervisor.stream(
    {
        "messages": [
            HumanMessage(
                content="Create a new page named Chad 222. write 3 sentences about anime character Chad from bleach in the same notion page."
            )
        ],
    },
    {"recursion_limit": 150},
):
    print(chunk)
    print("\n")

{'supervisor': {'messages': [HumanMessage(content='Create a new page named Chad 222. write 3 sentences about anime character Chad from bleach in the same notion page.', additional_kwargs={}, response_metadata={}, id='740ed984-1bed-4f97-80a2-0edf3efd6858'), AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_yqLcGNpp3I0yod92DHbIXhSS', 'function': {'arguments': '{}', 'name': 'transfer_to_notion_page_agent'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 443, 'total_tokens': 457, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-Bj6P9kwxKPDAcAZ5WYLR4TiPfUDLj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, name='supervisor', i